In [5]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import math

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

num_steps=28
num_inputs = 28
num_classes = 10
num_neurons = 128
num_layers = 3
batch_size = 500

graph = tf.Graph()
with graph.as_default():
    with tf.name_scope("graph_inputs"):
        X = tf.placeholder(tf.float32, [None, num_steps, num_inputs], name='input_placeholder')
        y = tf.placeholder(tf.float32, [None, num_classes], name='labels_placeholder')
        output_keep_prob = tf.placeholder_with_default(1.0, shape=(), name ="output_dropout")

    def build_lstm_cell(num_neurons, output_keep_prob):
        """Returns a dropout-wrapped LSTM-cell.
        See https://stackoverflow.com/a/44882273/2628369 for why this local function is necessary.
        Returns:
        tf.contrib.rnn.DropoutWrapper: The dropout-wrapped LSTM cell.
        """
        initializer = tf.contrib.layers.xavier_initializer()
        lstm_cell = tf.contrib.rnn.LSTMCell(num_units=num_neurons, initializer=initializer, forget_bias=1.0, state_is_tuple=True, name='LSTM_cell')
        lstm_cell_drop = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=output_keep_prob)
        return lstm_cell_drop
    
    with tf.name_scope("LSTM"):
        with tf.name_scope("Cell"):
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell([build_lstm_cell(num_neurons, output_keep_prob) for _ in range(num_layers)], state_is_tuple=True)
        with tf.name_scope("Model"):
            outputs, states = tf.nn.dynamic_rnn(cell=multi_layer_cell, inputs=X, swap_memory=False, time_major = False, dtype=tf.float32)#[Batch_size, time_steps, num_neurons]
        with tf.name_scope("Graph_Outputs"):
            outputs = tf.transpose(outputs, [1, 0, 2]) # [num_timesteps, batch_size, num_neurons]
            outputs = tf.gather(outputs, int(outputs.get_shape()[0]) - 1) # [batch_size, num_neurons]
        with tf.variable_scope('Softmax'):
            logits =  tf.layers.dense(inputs = outputs, units = num_classes, name="logits") #[Batch_size, num_classes]
        with tf.name_scope('Predictions'):
            predictions = tf.nn.softmax(logits, name="predictions")  #[Batch_size, num_classes]
        with tf.name_scope('Accuracy1'):
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy1 = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
        with tf.name_scope('Accuracy2'):
            accuracy2, accuracy_op = tf.metrics.accuracy(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1))
        with tf.name_scope('Loss'):
            xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
            loss = tf.reduce_mean(xentropy, name="loss")
        with tf.name_scope('Train'):
            optimizer= tf.train.AdamOptimizer(learning_rate=0.0001)
            trainer=optimizer.minimize(loss, name="training_op")

                        
#Batch accuracy
with tf.Session(graph = graph) as sess:
    tf.global_variables_initializer().run()
    total_batch = mnist.train.num_examples // batch_size
    for batch in range(total_batch):
        tf.local_variables_initializer().run()
        xBatch, yBatch = mnist.train.next_batch(batch_size)
        xBatch = xBatch.reshape((batch_size, num_steps, num_inputs))
        sess.run(trainer, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 0.5})
        miniBatchAccuracy1 = sess.run(accuracy1, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 0.5})
        print('[hand-written] Batch {} accuracy: {}'.format(batch, miniBatchAccuracy1))
        accuracy_op_val = sess.run(accuracy_op, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 0.5})
        miniBatchAccuracy2 = sess.run(accuracy2)
        print("[tf.metrics.accuracy] Batch {} accuracy: {}".format(batch, miniBatchAccuracy2))
    sess.close()

#overall accuracy, one batch at a time
with tf.Session(graph = graph) as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    avg_accuracy=0
    total_batch = mnist.train.num_examples // batch_size
    for batch in range(total_batch):
        xBatch, yBatch = mnist.train.next_batch(batch_size)
        xBatch = xBatch.reshape((batch_size, num_steps, num_inputs))
        sess.run(trainer, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 1})
        miniBatchAccuracy1 = sess.run(accuracy1, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 1})
        avg_accuracy += miniBatchAccuracy1 / total_batch
        
        accuracy_op_val = sess.run(accuracy_op, feed_dict={X: xBatch, y: yBatch, output_keep_prob: 1})
    miniBatchAccuracy2 = sess.run(accuracy2)    
        
        
    print('[hand-written] accuracy: {}'.format(avg_accuracy))
    print("[tf.metrics.accuracy] accuracy: {}".format(miniBatchAccuracy2))
    sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[hand-written] accuracy: 0.36461818157271897
[tf.metrics.accuracy] accuracy: 0.364618182182312
